# Cavity Dynamics

This notebook tests the ability of finesse to compute the eigenmode change of a linear cavity as a result of an optic being pitched or yawed.  
- Have analytical solution and finesse solutions (xaxis and fsig) for a linear cavity
- Have a general function which returns all mirrors and nodes (as pykat objects) from defined cavities

To do:
- Analytical 3 mirror
- Check if missing power from 1 sideband, right now a bandaid of 2x tilt applied
- Finesse 3 mirror
- define fsig method as a function using nodes and mirrors from OpticsandNodes()

Defined Functions:




In [1]:
import pykat
from pykat import finesse
from pykat.commands import *
import matplotlib.pyplot as plt
import numpy as np
import pykat.ifo.aligo as aligo
import pykat.ifo.aligo.plot as aplot
import pandas as pd

plt.rc('font', size= 10)
plt.rc('axes', titlesize= 10)


                                              ..-
    PyKat 1.2.1           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



## Two Mirror Analytical Solution

Here we look at the expected translation of a cavity axis using geometrical arguments. All formulas and how they arise can be found in Siegman chapter 19. The index for each variable indicates which optic we are referring to.

In [2]:
# Mirror Tilt:
theta_1 = 0                    # M1 tilt 
theta_2 = 1                     # M2 tilt 
w0 = 260.26206750103e-6                     # Beam waist
F = 625.17
lwav = 1064e-9

# Cavity parameters:
L = 0.25                        # Cavity Length
RcM2 = 0.5                    # Radius of curvature of M2 mirror =0.5 m
g1 = 1                         # g1 = 1 - L/RcM1 = 1 since RcM1 = infinity
g2 = 1 - L/RcM2                # g2 for M2 mirror
# print(g2)
# print('g1g2 =', g1*g2)         # Stability check

# Resulant translation 
delta_x1 = (g2/(1-g1*g2))*L*theta_1 + (1/(1- g1*g2))*L*theta_2    # Displacement on M1
delta_x2 = (1/(1-g1*g2))*L*theta_1 + (g1/(1-g1*g2))*L*theta_2     # Displacement on M2
delta_theta = (delta_x2 - delta_x1)/L                             # Angular displacement of optical axis

print('Results for analytical solutions:')
print('∆x1: ', delta_x1, ' m')
print('∆x2: ', delta_x2, ' m')
print(r'∆θ: ', delta_theta, ' rad')


Results for analytical solutions:
∆x1:  0.5  m
∆x2:  0.5  m
∆θ:  0.0  rad


## Two Mirror Finesse Solution

Now we use finesse to model the same cavity. The first string block is the definition of the linear cavity, the second is a number of detectors we plan to use (amplitude detectors inside the cavity to sense the first higher order mode.)

In [3]:
# Finesse Solution
basekat = finesse.kat()
basekat.verbose=False
basekat.parse(
"""
l laser 1 0 n0              # Laser 1W
s s0 0.5 n0 nmodin          # Distance from the laser to the first mirror M1
mod eo1 10M 0.3 3 pm nmodin nmodout
s s1 0.5 nmodout n1
pd1 pdh 10M -5 n1

#Cavity:
m M1 0.9999999999999999 0.0000000000000001 0 n1 n2            # Cavity mirror M1: R = 0.99, T = 0.01
s s_cav 0.25 n2 n3           # Cavity Length: L =  0.25 m
m M2 1 0 0 n3 n4            # Cavity mirror M2: R = 1, T = 0
attr M2 Rc 0.5              # Radius of curvature of M2 = 0.5 m
cav c1 M1 n2 M2 n3
trace 2
"""
)

basekat.maxtem = 3
basekat.trace = 2

# We need amplitude detectors for the HG00, HG01, and HG10 modes in order to measure translation
detkat = basekat.deepcopy()
detkat.noxaxis = True
detkat.parse(
"""
# Detectors:
pd circ n2
ad refl01 0 1 0 n1
ad refl10 1 0 0 n1

ad d00m1 0 0 0 n2
ad d00m2 0 0 0 n3

# y detectors:
ad d01m1 0 1 0 n2
ad d01m2 0 1 0 n3

# x detectors:
ad d10m1 1 0 0 n2
ad d10m2 1 0 0 n3

# beam measurements:
bp w0 x w0 n2
bp wm1 x w n2
bp wm2 x w n3
    
""")
# det = detkat.run()
# print (det.stdout)


### Two Mirror Finesse solution using Xaxis



In [139]:
sweepkat = deepcopy(detkat)

sweepkat.parse(
f"""
# locks
%set err pdh re
%lock z $err {-1./p[0]} 1e-14
%put* M1 phi $z

# Mirror tilt and axes
xaxis M1 ybeta lin 0 5e-9 1000
%put M1 xbeta $x1

yaxis re:im

""")

sweepout = sweepkat.run()
# print (sweepout.stdout)
# sweepout.plot()


In [140]:
out = sweepout
w0,wm1,wm2 = out['w0'].real,out['wm1'].real,out['wm2'].real
thetadiv = lwav/(w0*np.pi)

pcirc = (out['circ']).real
acirc = np.sqrt(pcirc)
abs00m1 = abs(out['d00m1'])
abs00m2 = abs(out['d00m2'])
# print (acirc,abs00m1,abs00m2)

## trying out leaving all complex until final calc:
m101 = out['d01m1']
m201 = out['d01m2']
m110 = out['d10m1']
m210 = out['d10m2']
Am101 = m101/abs00m1
Am201 = m201/abs00m1
Am110 = m110/abs00m1
Am210 = m210/abs00m1

# print ("M1 tilt")
# print ('dx m1',max(Am110.real*wm1))
# print ('dx m2',max(Am210.real*wm2))
# print ('dy m1',max(Am101.real*wm1),'| analytic:',delta_x1)
# print ('dy m2',max(Am201.real*wm2),'| analytic:',delta_x2)
# print ('dθ m1',max(Am101.imag*thetadiv))
# print ('dθ m2',max(Am201.imag*thetadiv))


# # plt.figure()
# # plt.plot(out.x*1e6,abs00m1)
# plt.figure()
# plt.plot(out.x*1e6,abs(Am110)*w0,label='M1 10') 
# plt.plot(out.x*1e6,abs(Am101)*w0,label='M1 01')
# plt.legend()

### Two Mirror Finesse solution using fsig

This solution uses fsig to dither each mirror and output the transfer function of tilt on M1 to beam displacement on cavity mirrors.   
To get the right results, I'm putting twice the amplitude on fsig as I do on the analytical function. Maybe I should not need to do that, if I count the mode amplitude on upper and lower sidebands created by the fsig command.  
 - [x] Counting upper and lower sidebands fixed this issue.

In [217]:
sigkat = basekat.deepcopy()
sigkat.noxaxis = True

sigkat.parse(
f"""
# Mirror tilt and axes
%fsig sig1 M1 xbeta 10 0 1
fsig sig1 M2 xbeta 10 0 1

yaxis re:im

# replace detectors with demodulated ones
pd circ n2
ad refl01 0 1 0 n1
ad refl10 1 0 0 n1

ad d00m1 0 0 0 n2
ad d00m2 0 0 0 n3

# y detectors:
ad d01m1 0 1 10 n2
ad d01m2 0 1 10 n3

# x detectors:
ad d10m1p 1 0 10 n2
ad d10m1n 1 0 -10 n2
ad d10m2p 1 0 10 n3
ad d10m2n 1 0 -10 n3

# beam measurements:
bp w0 x w0 n2
bp wm1 x w n2
bp wm2 x w n3
bp z1 x z n2
bp z2 x z n3
bp zr x zr n2
bp g1 x g n2
bp g2 x g n3

yaxis lin re:im

""")
sigout = sigkat.run()


out = sigout
w0,wm1,wm2 = out['w0'].real,out['wm1'].real,out['wm2'].real
thetadiv = lwav/(w0*np.pi)

pcirc = (out['circ']).real
acirc = np.sqrt(pcirc)
abs00m1 = out['d00m1']
abs00m2 = out['d00m2']
# print (acirc,abs00m1,abs00m2)

## trying out leaving all complex until final calc:
m100 = out['d00m1']
m200 = out['d00m2']
m101 = out['d01m1']
m201 = out['d01m2']
m110 = out['d10m1p']+out['d10m1n']
m210 = out['d10m2p']+out['d10m2n']
Am101 = m101/m100
Am201 = m201/m200
Am110 = m110/m100
Am210 = m210/m200


thetadiv = lwav/(w0*np.pi)
print ("M1 tilt")
print ('dx m1',Am110.real*wm1)
print ('dx m2',Am210.real*wm2)
print ('dy m1',Am101.real*wm1)
print ('dy m2',Am201.real*wm2)
print ('dθx m1',Am110.imag*thetadiv)
print ('dθx m2',Am210.imag*thetadiv)
print ('dθy m1',Am101.imag*thetadiv)
print ('dθy m2',Am201.imag*thetadiv)

M1 tilt
dx m1 0.5000000000000023
dx m2 -0.5000000000000031
dy m1 0.0
dy m2 0.0
dθx m1 -1.4261602315877164e-16
dθx m2 1.4142135623730963
dθy m1 0.0
dθy m2 0.0


In [184]:
out = sigout
w0,wm1,wm2 = out['w0'].real,out['wm1'].real,out['wm2'].real
thetadiv = lwav/(w0*np.pi)
m1div = lwav/(wm1*np.pi)
m2div = lwav/(wm2*np.pi)
gamma1 = (out['zr'].real**2-out['z1'].real**2)/out['zr'].real**2
gamma2 = (out['zr'].real**2-out['z2'].real**2)/out['zr'].real**2
# zR = (np.pi*w0**2)/lwav
# z = sigkat.s_cav.L
# gamma = (zR**2-z**2)/zR**2

pcirc = (out['circ']).real
acirc = np.sqrt(pcirc)
abs00m1 = out['d00m1']
abs00m2 = out['d00m2']
# print (acirc,abs00m1,abs00m2)

## trying out leaving all complex until final calc:
m100 = out['d00m1']
m200 = out['d00m2']
m101 = out['d01m1']
m201 = out['d01m2']
m110 = out['d10m1p']+out['d10m1n']
m210 = out['d10m2p']+out['d10m2n']
Am101 = m101/m100
Am201 = m201/m200
Am110 = m110/m100
Am210 = m210/m200


thetadiv = lwav/(w0*np.pi)
print ("M1 tilt")
print ('dx m1',Am110.real*wm1)
print ('dx m2',Am210.real*wm2)
print ('dy m1',Am101.real*wm1)
print ('dy m2',Am201.real*wm2)
print ('dθx m1',Am110.imag*m1div*gamma1)
print ('dθx m2',Am210.imag*m2div*gamma2)
print ('dθy m1',Am101.imag*m1div)
print ('dθy m2',Am201.imag*m2div)

M1 tilt
dx m1 0.5000000000000023
dx m2 -0.5000000000000031
dy m1 0.0
dy m2 0.0
dθx m1 -1.4261602315877164e-16
dθx m2 0.0
dθy m1 0.0
dθy m2 0.0


### How do the two compare?

Add details about error magnitudes in the two ways of calculating tilt to beam displacement..

#### Tilt on M1x
>Table on analytical result and xaxis, fsig results and deviations

#### Tilt on Mmn...

$\frac{1}{2}$

### Simple database example

The Finesse solution will have to be output in a well aranged, intuitive data structure. The pandas database lends itself well for storing, accessing and displaying the mirror to mirror data:

In [9]:
import pandas as pd

In [173]:
# how will the pandas db look?
# optic   |  

mirrorn = 3
mirrors = ['M1','M2','M3','M4']
brics = pd.DataFrame(np.random.randn(4, 4), columns=mirrors)  #replace with np.zeros((n,m))
# brics.index = mirrors
# print (brics)
# print (brics[['M1']])
# brics[['M1']]=np.transpose([0,0,0,0])
# df =  pd.DataFrame(index=list(zip(*wowitsadictionary[key]))[0])
# df['M1']=[1,2]#,3,4

In [176]:
brics['M1'][3]

0.6096234744443385

# Three Mirror Cavity Dynamics

The next few cells serve as the beginning to generalizing this tool to work on any kat file containing any number of cavities (where in alpha any number of cavities has an upper limit of 1)

 - [ ] Make separate function for placing the middle mirror

In [4]:
# ============ Triangular Cavity ================
d = 0.30
R = 37.8
L = 10.05
l0 = np.sqrt(d**2 + L**2)
phi = 89/2 #89.14490260373327/2

tri = finesse.kat()
tri.parse(
f'''
l l0 1 0 n0
s s0 0 n0 nMC1in
bs MC1 0.99 0.01 0 -{phi} nMC1in nMC1refl nMC1trans nMC1ret
s sMC1toMC3 {d} nMC1trans nMC3in
bs MC3 1 0 0 {phi} nMC3in nMC3refl nMC3trans nMC3ret
s sMC3toMC2 {l0} nMC3refl nMC2in
bs MC2 1 0 0 1 nMC2in nMC2refl nMC2trans dump
s sMC2toMC1 {l0} nMC2refl nMC1ret
attr MC2 Rc {R}
cav TMC MC2 nMC2refl MC2 nMC2in
pd refl nMC1in
'''
)

In [178]:
ligo = aligo.make_kat("design", preserveConstants=False) 
ligo = aligo.setup(ligo)

In [179]:
# ligo.getBlocks()
ligo.removeBlock('locks')

### Gathering optics to be dithered and associated nodes

This function takes a kat file, reads and stores cav command names to be used as keys then uses the command names to find all intracavity nodes using trace 128 and the stdout string.  
Nodes are associated to their respective cav command.  
Nodes are cycled through to find the optical components attached to them. It stores one node per optical component in a dictionary. Optics are checked to be "real" optics (as opposed to AR surfaces, etc) by checking T and R.  
The dictionary is structure:  
> { 'cav command 1' : [ [ optic1 , node1 ] , [ optic2 , node2 ] , ... ] , 'cav command 2' : ... }

In [5]:

def OpticsandNodes(kat,verbose=True):
    
    import pykat.commands as pkc
    
    kat = kat.deepcopy()
    kat.noxaxis = True
    kat.verbose = False
    kat.trace = 128

    cavs = kat.getAll(pkc.cavity)
    mirs = []
    if len(cavs)==0:
        raise Exception("no cav commands defined.")

    pdnode = kat.nodes.getComponentNodes(kat.getAll(pkc.Component)[0])[0].name
    if len(kat.detectors)==0:
        lasers = kat.getAll(pkc.laser)
        if len(lasers)==0:
            raise Exception("no laser in model")
        kat.parse(f"pd Pin {lasers[0].nodes[0].name}")

    o = kat.run()
#     print (o.stdout)
    allLines = o.stdout


    lines = o.stdout.splitlines()

    for c,cav in enumerate(cavs):
        cavname = str(cav.name)
        for i,l in enumerate(lines):
    #         print (i)
            cavfind = l.find(cavname)
            if cavfind != -1:
                cn = []
                cav = l.split()[1][:-1]
    #             print (cav)
                if lines[i+1].find('found the cavity nodes:')!=1:
                    nodelines = lines[i+1].split()[4:]
                if lines[i+2].find('... and back:')!=-1:
                    nodelines.extend(lines[i+2].split()[3:])
                comps = []
    #             print (nodelines)
                for j,nl in enumerate(nodelines):
                    if len(nodelines)>2 and j==0: continue   ## don't look at the refl node of this mirror
#                     print (len(nodelines))
                    n = nl[:nl.find("(")]
                    nn = getattr(kat.nodes,n)
                    linked = kat.nodes.getNodeComponents(nn)
                    for comp in linked:
                        if isinstance(comp, pkc.mirror) or isinstance(comp, pkc.beamSplitter):
                            if comp not in comps:
                                if (comp.T!=None and comp.T<0.5) or (comp.R!=None and comp.R>0.5):
                                    comps.append(comp)
                                    cn.append([comp,nn])
    #                                 print (comp,comp.R,comp.T)
                mirs.append(cn)
                break

    if verbose:
        for i,j in zip(cavs,mirs):
            print (i)
            for k in j:
                print (k[0],k[1])
            print ()

    dict1 = {}
    for i,j in zip(cavs,mirs):
        a = str(i.name)
        dict1[a]=j
    return dict1

wowitsadictionary = OpticsandNodes(tri,verbose=False)
print (wowitsadictionary)
# cav TMC MC2 nMC2refl MC2 nMC2in

{'TMC': [[<pykat.components.beamSplitter_25 (MC1) at 0x2487089dd88>, <Node (nMC1ret) at 0x248708bd088>], [<pykat.components.beamSplitter_27 (MC3) at 0x248708a7c88>, <Node (nMC3in) at 0x248708bd148>], [<pykat.components.beamSplitter_29 (MC2) at 0x248708aebc8>, <Node (nMC2in) at 0x248708bd448>]]}


### Dithering optics

 - [x] This should be its own function
 - Adding new function to calculate gamma and apply to angles

key : 'TMC'  
MC2 - nMC2in
MC1 - nMC1ret  
MC3 - nMC3in  


In [6]:
# print (wowitsadictionary)

def shakeIt(kat,dictionary,precision=10,limit=1e-10,verbose=True,show_all=False,getkat=False):
    ## Stuff it needs to work
    import pandas as pd
    import pykat.commands as pkc
    import numpy as np
    
    pd.set_option('precision', precision)
    d = dictionary
    waveL = kat.lambda0
    
    ## Copy the kat, don't want to modify it
    k = kat.deepcopy()
    k.verbose = False
    k.noxaxis = True
    k.maxtem = 1 #why do I need to set maxtem for triangular but not for linear?

    ## looping through cavities
    dfs = {}
    for key in d:
        ## Place bps in x and y to measure the beam waist in that cavity
        w0node = d[key][0][1]
        k.parse(f"""
        bp w0x x w0 {w0node}
        bp w0y y w0 {w0node}
        yaxis re:im
        """)
        fdither = 10
        
        ## Loop through mirrors in cavity

        for mir,node in d[key]:
            ## Maximize mirror R to keep all those juicy modes in
            k.components[mir.name].R = 0.9999999999999999
            k.components[mir.name].T = 0.0000000000000001
            k.components[mir.name].L = 0.
            ## Add ad00, ad10 (X) and ad01 (Y) detectors for each mirror
            k.parse(f"""
            %%% --- {mir.name}:
            % - amplitude detectors
            ad d00{mir.name} 0 0 0 {node.name}
            ad d10{mir.name}p 1 0 {fdither} {node.name}
            ad d01{mir.name}p 0 1 {fdither} {node.name}
            ad d10{mir.name}n 1 0 {-fdither} {node.name}
            ad d01{mir.name}n 0 1 {-fdither} {node.name}
            % - beam parameter
            bp w{mir.name}x x w {node.name}
            bp w{mir.name}y y w {node.name}
            """)
            
        ## Loop through mirrors in cavity to check that ad is looking at light
        vibe_check = k.run()
        for mir,node in d[key]:
            if abs(vibe_check[f'd00{mir.name}'])==0:
                k.parse(f"""
                %%% --- {mir.name}:
                % - amplitude detectors
                ad d00{mir.name} 0 0 0 {node.name}*
                ad d10{mir.name}p 1 0 {fdither} {node.name}*
                ad d01{mir.name}p 0 1 {fdither} {node.name}*
                ad d10{mir.name}n 1 0 {-fdither} {node.name}*
                ad d01{mir.name}n 0 1 {-fdither} {node.name}*
                % - beam parameter
                bp w{mir.name}x x w {node.name}*
                bp w{mir.name}y y w {node.name}*
                """)
        vibe_check = k.run()
        if abs(vibe_check[f'd00{mir.name}'])==0:
            raise Exception("it's broken :(")
        
        ## create dataframe to store the tilt and trans response
        ind=[]
        for n in (list(zip(*d[key]))[0]):
            ind.extend([f"{n} shift [m/rad]",f"{n} tilt [rad/rad]"])
        df = pd.DataFrame(index=ind)
        
        ## loop through mirrors in cavity to apply fsig on each
        for mir,node in d[key]:
            
            ## Apply xbeta
            kx = k.deepcopy()
            kx.parse(f"""
            fsig sig1 {mir.name} xbeta {fdither} 0 1
            """)
            o = kx.run()
            [print (f"###Code to tilt {mir.name}x:###\n",kx) if getkat else None]
            thetadiv = (1064e-9)/(o[f'w{mir.name}x'].real*np.pi)
            ## get wavelength from finesse file
            
            
            #loop through mirrors in cavity to store response in x
            response=[]
            for m,n in d[key]:
                m00 = o[f'd00{m.name}']
                m10 = o[f'd10{m.name}p']+o[f'd10{m.name}n']
                wm = o[f'w{m.name}x'].real
                Am10 = m10/m00
                
                ## check if response is close to numerical error, throw out if it is
                if abs(Am10.real*wm)>1e-16: 
#                     response.append(Am10.real*wm)
                    if isinstance(m, pkc.mirror):
                        response.append(Am10.real*wm)
                    elif isinstance(m, pkc.beamSplitter):
                        response.append((Am10.real*wm)/np.cos(np.radians(k.components[m.name].alpha.value)))
#                         print (1/np.cos(np.radians(k.components[m.name].alpha.value)),k.components[m.name].alpha.value)
                else:
                    response.append(0.)
                    
                if abs(Am10.imag*thetadiv)>limit: 
                    response.append(Am10.imag*thetadiv)
                else:
                    response.append(0.)
                
            df[mir.name+'x'] = response
            
            ## Apply ybeta
            ky = k.deepcopy()
            ky.parse(f"""
            fsig sig1 {mir.name} ybeta {fdither} 0 1
            """)
            o = ky.run()
            [print (f"###Code to tilt {mir.name}y:###\n",ky) if getkat else None]
            thetadiv = (1064e-9)/(o[f'w{mir.name}y'].real*np.pi)
            
            ## loop through mirrors in cavity to store response in y
            response=[]
            for m,n in d[key]:
                m00 = o[f'd00{m.name}']
                m01 = o[f'd01{m.name}p']+o[f'd01{m.name}n']
                wm = o[f'w{m.name}y'].real
                Am01 = m01/m00
                
                ## check if response is close to numerical error, throw out if it is
                if abs(Am01.real*wm)>1e-16: 
                    response.append(Am01.real*wm)
                else:
                    response.append(0.)
                    
                if abs(Am01.imag*thetadiv)>1e-15: 
                    response.append(Am01.imag*thetadiv)
                else:
                    response.append(0.)
                    
            df[mir.name+'y'] = response
        dfs[key]=df
        if verbose: print (f'Cavities in the dictionary: {mdict.keys()}')
        if show_all:
            for k in dfs.keys():
                display(dfs[k])
    return dfs


In [10]:
# print (wowitsadictionary)

def shakeIt2(kat,dictionary,precision=3,limit=1e-10,verbose=True,show_all=False,getkat=False):
    ## Stuff it needs to work
    import pandas as pd
    import pykat.commands as pkc
    import numpy as np
    
    pd.set_option('precision', precision)
    d = dictionary
    waveL = kat.lambda0
    
    ## Copy the kat, don't want to modify it
    k = kat.deepcopy()
    k.verbose = False
    k.noxaxis = True
    k.maxtem = 1 

    ## looping through cavities
    dfs = {}
    for key in d:
        ## Place bps in x and y to measure the beam waist in that cavity
        w0node = d[key][0][1]
        k.parse(f"""
        bp w0x x w0 {w0node}
        bp w0y y w0 {w0node}
        bp zrx x zr {w0node}
        bp zry y zr {w0node}
        yaxis re:im
        """)
        fdither = 10
        
        ## Loop through mirrors in cavity

        for mir,node in d[key]:
            ## Maximize mirror R to keep all those juicy modes in
            k.components[mir.name].R = 0.9999999999999999
            k.components[mir.name].T = 0.0000000000000001
            k.components[mir.name].L = 0.
            ## Add ad00, ad10 (X) and ad01 (Y) detectors for each mirror
            k.parse(f"""
            %%% --- {mir.name}:
            % - amplitude detectors
            ad d00{mir.name} 0 0 0 {node.name}
            ad d10{mir.name}p 1 0 {fdither} {node.name}
            ad d01{mir.name}p 0 1 {fdither} {node.name}
            ad d10{mir.name}n 1 0 {-fdither} {node.name}
            ad d01{mir.name}n 0 1 {-fdither} {node.name}
            % - beam parameter
            bp w{mir.name}x x w {node.name}
            bp w{mir.name}y y w {node.name}
            bp z{mir.name}x x z {node.name}
            bp z{mir.name}y y z {node.name}
            bp g{mir.name}x x g {node.name}
            bp g{mir.name}y y g {node.name}
            """)

        ## Loop through mirrors in cavity to check that ad is looking at light
#         k.trace = 2
        vibe_check = k.run()
#         print (vibe_check.stdout)
        for mir,node in d[key]:
            if abs(vibe_check[f'd00{mir.name}'])==0:
                k.parse(f"""
                %%% --- {mir.name}:
                % - amplitude detectors
                ad d00{mir.name} 0 0 0 {node.name}*
                ad d10{mir.name}p 1 0 {fdither} {node.name}*
                ad d01{mir.name}p 0 1 {fdither} {node.name}*
                ad d10{mir.name}n 1 0 {-fdither} {node.name}*
                ad d01{mir.name}n 0 1 {-fdither} {node.name}*
                % - beam parameter
                bp w{mir.name}x x w {node.name}*
                bp w{mir.name}y y w {node.name}*
                bp z{mir.name}x x z {node.name}*
                bp z{mir.name}y y z {node.name}*
                """)
        vibe_check = k.run()
        if abs(vibe_check[f'd00{mir.name}'])==0:
            raise Exception("it's broken :(")
        
        ## create dataframe to store the tilt and trans response
        shift_ind = []
        tilt_ind = []
        for n in (list(zip(*d[key]))[0]):
            shift_ind.extend([f"{n} shift [m/rad]"])
            tilt_ind.extend([f"Tilt [rad/rad]"])
        df_shift = pd.DataFrame(index=shift_ind)
        df_tilt = pd.DataFrame(index=tilt_ind)
        
        #######
        ## loop through mirrors in cavity to apply fsig on each
        #######
        for mir,node in d[key]:
            
            ## Apply xbeta
            ########
            kx = k.deepcopy()
            kx.parse(f"""
            fsig sig1 {mir.name} xbeta {fdither} 0 1
            """)
            o = kx.run()
            [print (f"###Code to tilt {mir.name}x:###\n",kx) if getkat else None]
            
            
            
            ## loop through mirrors in cavity to store response in x
            #######
            shifts=[]
            tilts=[]
            for m,n in d[key]:
                m00 = o[f'd00{m.name}']
                m10 = o[f'd10{m.name}p']+o[f'd10{m.name}n']
                wm = o[f'w{m.name}x'].real
                Am10 = m10/m00
                gamma = (o['zrx'].real**2+o[f'z{m.name}x'].real**2)/o['zrx'].real**2
                thetadiv = waveL/(o[f'w{m.name}x'].real*np.pi)
                div0 = waveL/(o["w0x"].real*np.pi)
                psi = o[f"g{m.name}x"]
                
                ## check if response is close to numerical error, throw out if it is
                if abs(Am10.real*wm)>1e-16: 
                    if isinstance(m, pkc.mirror):
                        shifts.append(Am10.real*wm)
                    ## Corrects for beamsplitter angle
                    elif isinstance(m, pkc.beamSplitter):
                        shifts.append((Am10.real*wm)/np.cos(np.radians(k.components[m.name].alpha.value)))
                else:
                    shifts.append(0.)
                    
                if abs((Am10*np.exp(1j*psi)).imag*div0)>limit: 
                    tilts.append((Am10*np.exp(1j*psi)).imag*div0)
                else:
                    tilts.append(0.)
                
            df_shift[mir.name+'x'] = shifts
            df_tilt[mir.name+'x'] = tilts
            
            ## Apply ybeta
            ########
            ky = k.deepcopy()
            ky.parse(f"""
            fsig sig1 {mir.name} ybeta {fdither} 0 1
            """)
            o = ky.run()
            [print (f"###Code to tilt {mir.name}y:###\n",ky) if getkat else None]
            
            ## loop through mirrors in cavity to store response in y
            #######
            shifts=[]
            tilts=[]
            for m,n in d[key]:
                m00 = o[f'd00{m.name}']
                m01 = o[f'd01{m.name}p']+o[f'd01{m.name}n']
                wm = o[f'w{m.name}y'].real
                Am01 = m01/m00
                gamma = (o['zrx'].real**2+o[f'z{m.name}x'].real**2)/o['zrx'].real**2
                thetadiv = waveL/(o[f'w{m.name}x'].real*np.pi)
                div0 = waveL/(o["w0y"].real*np.pi)
                psi = o[f"g{m.name}y"]
                
                ## check if response is close to numerical error, throw out if it is
                if abs(Am01.real*wm)>1e-16: 
                    shifts.append(Am01.real*wm)
                else:
                    shifts.append(0.)
                    
                if abs((Am01*np.exp(1j*psi)).imag*div0)>limit: 
                    tilts.append((Am01*np.exp(1j*psi)).imag*div0)
                else:
                    tilts.append(0.)
                    
            df_shift[mir.name+'y'] = shifts
            df_tilt[mir.name+'y'] = tilts
        dfs[key] = df_shift.append(df_tilt.iloc[0])
        if verbose: print (f'Cavities in the dictionary: {mdict.keys()}')
        if show_all:
            for k in dfs.keys():
                display(dfs[k])
    return df_shift, df_tilt, dfs


In [11]:
k = basekat
mdict = OpticsandNodes(k,verbose=False)
dfs,dft,df = shakeIt2(k,mdict,show_all=False)
display(dfs)
display(dft)
display(df['TMC'])



Cavities in the dictionary: dict_keys(['c1'])


,M1x,M1y,M2x,M2y
M1 shift [m/rad],-0.25,0.25,0.5,-0.5
M2 shift [m/rad],0.50,0.50,-0.5,-0.5


,M1x,M1y,M2x,M2y
Tilt [rad/rad],1.0,-1.0,0.0,0.0
Tilt [rad/rad],1.0,1.0,0.0,0.0


KeyError: 'TMC'

In [405]:
k = tri
mdict = OpticsandNodes(k,verbose=True)
dfs2 = shakeIt2(k,mdict,show_all=True,Mnot=False)

TMC
MC1 nMC1ret
MC3 nMC3in
MC2 nMC2in

Cavities in the dictionary: dict_keys(['TMC'])


In [221]:
k = tri
mdict = OpticsandNodes(k,verbose=False)
dfs = shakeIt(k,mdict,show_all=False)

Cavities in the dictionary: dict_keys(['TMC'])


In [476]:
cavdf = df['TMC'].copy()
limit = 1e-10
nk = []
for key in cavdf.keys():
    if 'y' in key:
        nk.append(key)
m1 = 'MC1y'
m2 = 'MC3y'
newdf = cavdf[nk].copy()
newdf[f'{m1}_{m2}_C'] = newdf.apply(lambda row: ((row[f'{m1}']+row[f'{m2}'])/2
                                                if abs(row[f'{m1}']+row[f'{m2}'])/2>limit
                                                else 0.0),
                                    axis=1)
newdf[f'{m1}_{m2}_D'] = newdf.apply(lambda row: ((row[f'{m1}']-row[f'{m2}'])/2
                                                if abs(row[f'{m1}']-row[f'{m2}'])/2>limit
                                                else 0.0),
                                    axis=1)
newdf

,MC1y,MC3y,MC2y,MC1y_MC3y_C,MC1y_MC3y_D
MC1 shift [m/rad],19.688,-19.899,-37.8,-0.105,19.794
MC3 shift [m/rad],19.899,-19.688,-37.8,0.105,19.794
MC2 shift [m/rad],26.965,-26.965,-37.8,0.000,26.965
Tilt [rad/rad],0.724,-0.703,0.0,0.010,0.713


In [486]:
def dofs(df,m1,m2,limit=1e-10):
    """ df is a cavity dataframe output by shakeIt(). m1 and m2 are mirrors of interest for common and differential tilt. 
    There isn't a need to specify x or y (for example, only need to specify MC1 and MC3 not MC1x...)"""
    
    ## Running for y dof (specified by ybeta)
    nky = []
    for key in df.keys():
        if 'y' in key:
            nky.append(key)
    ydof = df[nky].copy()
    ydof[f'{m1}y_{m2}y_D'] = ydof.apply(lambda row: ((row[f'{m1}y']+row[f'{m2}y'])/2
                                                    if abs(row[f'{m1}y']+row[f'{m2}y'])/2>limit
                                                    else 0.0),
                                        axis=1)
    ydof[f'{m1}y_{m2}y_C'] = ydof.apply(lambda row: ((row[f'{m1}y']-row[f'{m2}y'])/2
                                                    if abs(row[f'{m1}y']-row[f'{m2}y'])/2>limit
                                                    else 0.0),
                                        axis=1)
    
    
    ## Running for x dof (specified by xbeta)
    nkx = []
    for key in df.keys():
        if 'x' in key:
            nkx.append(key)
    xdof = df[nkx].copy()
    xdof[f'{m1}x_{m2}x_D'] = xdof.apply(lambda row: ((row[f'{m1}x']+row[f'{m2}x'])/2
                                                    if abs(row[f'{m1}x']+row[f'{m2}x'])/2>limit
                                                    else 0.0),
                                        axis=1)
    xdof[f'{m1}x_{m2}x_C'] = xdof.apply(lambda row: ((row[f'{m1}x']-row[f'{m2}x'])/2
                                                    if abs(row[f'{m1}x']-row[f'{m2}x'])/2>limit
                                                    else 0.0),
                                        axis=1)
    
    
    
    
    return xdof.transpose(),ydof.transpose()


cavdf = df['TMC'].copy()
# cavdf2 = dfs2['TMC'].copy()
m1 = 'MC1'
m2 = 'MC3'
xdof,ydof = dofs(cavdf,m1,m2)
# xdof2,ydof2 = dofs(cavdf2,m1,m2)
display (xdof)
# display (xdof2)
display (ydof)
# display (ydof2)

,MC1 shift [m/rad],MC3 shift [m/rad],MC2 shift [m/rad],Tilt [rad/rad]
MC1x,-14.308,13.885,0.206,0.995
MC3x,13.885,-14.308,0.206,-1.005
MC2x,0.288,0.288,-13.981,1.370
MC1x_MC3x_D,-0.211,-0.211,0.206,-0.005
MC1x_MC3x_C,-14.097,14.097,0.000,1.000


,MC1 shift [m/rad],MC3 shift [m/rad],MC2 shift [m/rad],Tilt [rad/rad]
MC1y,19.688,19.899,26.965,0.724
MC3y,-19.899,-19.688,-26.965,-0.703
MC2y,-37.800,-37.800,-37.800,0.000
MC1y_MC3y_D,-0.105,0.105,0.000,0.010
MC1y_MC3y_C,19.794,19.794,26.965,0.713



<img src="capture.png" alt="fumiko" width="600"/>

 - [x] Start with normal R,T and have function set it to max R
 - ~~separate dictionaries for x and y~~
 - [x] where do i place angular freedom?
 - write up notebook markdown intro
 - paper skeleton
 - [x] ~~single function all together~~ in tools
 - [x] ~~Is the bandaid ok~~ for the most part, but add a loop to check for no light and flip pds if so
 - ~~common and differential? might need user input?~~
 - ~~multicav file processing (how do coupled cavities behave?)~~ will attempt in Finesse 3
 - how to consider fsig phase in terms of direction of tilt
 - 